In [ ]:
import langchain
langchain.__version__

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain.tools import tool

llm = ChatOllama(model="llama3.2:latest", temperature=0)


In [ ]:
@tool("admission_seat_count", description="This tells the student count of department wise")
def admission_seat_count():
    return {
        "electronics":23,
        "computer_science":45,
        "mechanical":60,
        "english":2
    }

@tool("faculty_openings", description="this tells the no. of position opening for faculty based on the major degree")
def faculty_openings():
    return {
        "technology":3,
        "engineering":5,
        "master":4
    }

In [ ]:
agent = create_agent(model=llm, system_prompt="You are an intelligent assistant help us on the questions related to the college admissions and faculty openings, please precise with your answer no need to share other information.", tools=[admission_seat_count, faculty_openings])

response = agent.invoke({
    "messages":[
        HumanMessage(content="Tell me no of student seats available for english? and no of professors positions open for engineering")
    ]
}
)
print(response["messages"][-1].content)
agent
#result = llm.invoke("Tell me details on python jupiter notebook in 2 lines?")
#result.content_blocks

In [ ]:
agent1 = create_agent(model=llm, system_prompt="You are an intelligent ai assistant to answer user queries")

# Correct way to stream with agents
response = agent1.stream(
    {"messages": [HumanMessage(content="tell me on langchain middleware in 200 words?")]},
    stream_mode="messages"
)
print(response)
# print(type(response))
# for chunk, metadata in response:
#     # chunk is the message, metadata contains info about which node produced it
#     if hasattr(chunk, 'content') and chunk.content:
#         print(chunk.content, end="", flush=True)

In [ ]:
# Alternative streaming approaches

print("\n" + "="*60)
print("Method 1: Stream mode 'values' (recommended for agents)")
print("="*60)

response = agent1.stream(
    {"messages": [HumanMessage(content="What is LangChain in 50 words?")]},
    stream_mode="values"
)

print(response)
# for state in response:
#     # Get the last message from the state
#     if "messages" in state and state["messages"]:
#         last_msg = state["messages"][-1]
#         if hasattr(last_msg, 'content'):
#             print(last_msg.content)

# print("\n" + "="*60)
# print("Method 2: Stream mode 'updates' (shows incremental updates)")
# print("="*60)
#
# response = agent1.stream(
#     {"messages": [HumanMessage(content="Explain Python in 30 words?")]},
#     stream_mode="updates"
# )
#
# for update in response:
#     print(update)

# print("\n" + "="*60)
# print("Method 3: Direct LLM streaming (no agent wrapper)")
# print("="*60)
#
# # If you just want to stream LLM responses without agent logic
# for chunk in llm.stream("What is machine learning in 40 words?"):
#     print(chunk.content, end="", flush=True)

"""
STREAMING WITH LANGCHAIN AGENTS - EXPLANATION

The issue: agent.stream() with stream_mode="messages" returns TUPLES, not message objects!

Format: (message_chunk, metadata_dict)

Common stream_mode options:
1. "messages" - Returns (chunk, metadata) tuples for each message chunk
2. "values" - Returns complete state after each step (easier to use)
3. "updates" - Returns only the updates/changes to state

Best practices:
- For token-by-token streaming: Use stream_mode="messages" and unpack tuples
- For step-by-step results: Use stream_mode="values"
- For direct LLM streaming (no agent): Use llm.stream() directly
"""